In [ ]:
import matplotlib.pyplot as plt
import types
import numpy as np
import sklearn
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from scipy.stats import norm
from __future__ import division
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
## split_size
split_size=0.3

### Author C. Joshi
### The dataset has been obtained from
##https://data.gov.uk/dataset/statistics_on_obesity_physical_activity_and_diet_england

### What this dataset is about ?
#### This statistical report presents a range of information on obesity, physical activity 
#and diet, drawn together from a variety of sources.

#The topics covered include:

#Overweight and obesity prevalence among adults and children

#Physical activity levels among adults and children

#Trends in purchases and consumption of food and drink and energy intake

#Health outcomes of being overweight or obese.

#The annual compendium report presents new analyses by NHS Digital which 
#consists of statistics on the number of NHS hospital admissions attributable
#to obesity, the number of NHS hospital admissions attributable to a procedure 
#of 'Bariatric Surgery', and the number of prescription items provided in primary
#care for the treatment of obesity. The report focuses on England. 
#The latest available hospital admissions data is for 2015/16. The latest available 
#prescribing data is for 2016.


### STEP ONE :READING THE DATASET 
space=100
workbook=pd.ExcelFile('obes-phys-acti-diet-eng-2017-tab.xlsx')
list_sheets=workbook.sheet_names
df1 = pd.read_excel('obes-phys-acti-diet-eng-2017-tab.xlsx',\
sheetname=list_sheets[1])
df1_clean=df1[df1['Unnamed: 2'].apply(lambda x: type(x)==int)]
df1_clean=df1_clean.dropna(axis=1, how='all')
df1_clean.index=np.arange(np.shape(df1_clean)[0])
df1_clean.columns=['Year','All persons','Male','Female']
#df1_clean.index=df1_clean['Year']
#del df1_clean['Year']
#print df1_clean
df1_clean_rs = pd.melt(df1_clean.loc[:,['Year','All persons','Male','Female']], id_vars="Year", var_name="sex", \
                    value_name="Number of Obesity cases")
#print df1_clean
fig, ax = plt.subplots()
sns.set()
sns.barplot(x='Year', y='Number of Obesity cases', hue='sex', data=df1_clean_rs)
df1_clean.plot(x='Year',y='Male',color='green',ax=ax)
df1_clean.plot(x='Year',y='Female',color='red',ax=ax)
df1_clean.plot(x='Year',y='All persons',color='blue',ax=ax)
plt.xlabel("Year")
plt.ylabel("Number of Obesity cases")
plt.title("Finished Admission Episodes with a primary diagnosis of obesity by gender")
plt.show()
print "The trend suggest that the number of male obesity cases seem to"
print "observe an approximately constant trend"
print "while the obesity cases among the female cases \
saw a non-monotonic behaviour."
print "+"*space
print "FITTING PROECDURE"
print "+"*space



df1_clean['dummy_year']=1+np.arange(len(df1_clean['Year']))
grpedyear=zip(df1_clean['dummy_year'].tolist(),df1_clean['Year'].tolist())
X_train,X_test,y_train,y_test = train_test_split(df1_clean.loc[:,['dummy_year','Year']],\
df1_clean['Male'][:,np.newaxis],test_size=split_size)





fet_lst=['Male','Female']
poly_ord=[2,3,4]
pntsmax=100
strdata=[]
strpolydata=[]
streqn=[]
for out_indx in range(len(fet_lst)):
    for in_indx in range(len(poly_ord)):
        x = np.array(df1_clean['dummy_year'])
        strx=list(x)
        data=np.array(df1_clean[fet_lst[out_indx]])
        strdata.append(list(data))
        poly_params = np.polyfit(x, data, poly_ord[in_indx]) # Fit the data with a nth degree polynomial
        poly_ft = np.poly1d(poly_params)      # Construct the polynomial
        streqn.append(list(poly_ft.c))
        xPoly = np.linspace(0, max(x), pntsmax)  # Generate 100 x-coordinates from 0 to max(x)
        strpolyx=list(xPoly)
        strpolydata.append(list(poly_ft(xPoly)))  # Use the polynomial to calculate the y-coordinates
#strx=np.reshape(strx,(len(fet_lst)*len(poly_ord),len(df1_clean['dummy_year'])))
strdata=np.reshape(strdata,(len(fet_lst)*len(poly_ord),len(df1_clean['dummy_year'])))
#strpolyx=np.reshape(strpolyx,(len(fet_lst)*len(poly_ord),len(xPoly)))
strpolydata=np.reshape(strpolydata,(len(fet_lst)*len(poly_ord),len(xPoly)))
streqn=np.reshape(streqn, (len(fet_lst),len(poly_ord)))



f, ax = plt.subplots(1, 1)
sns.set()
fet=0
ax.plot(strx, strdata[fet,:], 'o',label=str(fet_lst[0])+" obese cases")
ax.plot(strpolyx, strpolydata[fet,:], '-*',label="Fit of polynomial order "+str(poly_ord[fet]))
ax.plot(strpolyx, strpolydata[fet+1,:], '-.',label="Fit of polynomial order "+str(poly_ord[fet+1]))
ax.plot(strpolyx, strpolydata[fet+2,:], '-o',label="Fit of polynomial order "+str(poly_ord[fet+2]))

fet=len(poly_ord)
ax.plot(strx, strdata[fet,:], 'o',label=str(fet_lst[1])+" obese cases")
ax.plot(strpolyx, strpolydata[fet,:], '-*',label="Fit of polynomial order "+str(poly_ord[fet-fet]))
ax.plot(strpolyx, strpolydata[fet+1,:], '-.',label="Fit of polynomial order "+str(poly_ord[fet+1-fet]))
ax.plot(strpolyx, strpolydata[fet+2,:], '-o',label="Fit of polynomial order "+str(poly_ord[fet+2-fet]))
ax.set_xticklabels(df1_clean.Year)
plt.xlabel("Years ")
plt.ylabel("Number of cases")             
#ax.set_title(title)
ax.legend()
plt.show()



print "+"*space
### Adding more features to the dataframe:
df2 = pd.read_excel('obes-phys-acti-diet-eng-2017-tab.xlsx',\
sheetname=list_sheets[2])
df2_clean=df2[df2['Unnamed: 2'].apply(lambda x: type(x)==int)]
df2_clean=df2_clean.dropna(axis=1, how='all')
df2_clean.index=np.arange(np.shape(df2_clean)[0])
df2_clean.columns=['Year','All persons','Under-16','16-24',\
                  '25-34','35-44','45-54','55-64','65-74','75-over']
del df2_clean['All persons']
df_1_2_clean= pd.merge(df1_clean, df2_clean, on='Year', how='outer')

print "WE WILL NOW ANALYSE THE OBESITY VARIATION IN DIFFERENT AGE GROUPS OVER TIME"
print "+"*space
age_grp_list=list(df2_clean.columns[2:])



agegrplist=list(df_1_2_clean.columns)[4:]
print '+'*space
str_age_df=pd.DataFrame()
poly_order=1
print "POLYNOMIAL REGRESSION OF ORDER "+str(poly_order)
print '+'*space

poly = PolynomialFeatures(degree=poly_order)
ply_obes_df = poly.fit_transform(df_1_2_clean[agegrplist[0]][:,np.newaxis])
target_feature_names = ['x'.join(['{}^{}'.format(pair[0],pair[1]) \
                                  for pair in tuple if pair[1]!=0]) \
                        for tuple in [zip(agegrplist,p) for p in poly.powers_]]
temp_df = pd.DataFrame(ply_obes_df, columns = target_feature_names)
temp_df.rename(columns={'': 'Coeff'}, inplace=True)
fit_colmnslst=list(temp_df.columns)
str_age_df=pd.merge(df_1_2_clean[list(df_1_2_clean.columns)[0:]], temp_df, left_index=True,right_index=True)



### Polynmoial Regression  
index=0
by_age=agegrplist[1:]
fit_colmnslst
poly_X_train,poly_X_test,poly_y_train,poly_y_test \
    = train_test_split(str_age_df[fit_colmnslst],\
str_age_df[by_age[len(by_age)-1]][:,np.newaxis],test_size=split_size)
    
# Create linear regression object
#clf = linear_model.Lasso()
regr = linear_model.LinearRegression()

# Train the model using the training sets
#regr.fit(poly_X_train, poly_y_train)
lin_model=regr.fit(poly_X_train,poly_y_train)
accuracy = regr.score(poly_X_test,poly_y_test)

# Make predictions using the testing set
poly_y_pred = regr.predict(poly_X_test) 



print "Variance score: %1f"%\
r2_score(poly_y_test, poly_y_pred)
print '+'*space
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    
print("Mean squared error: %.2f"% \
mean_squared_error(poly_y_test, poly_y_pred))
print "The coefficients are :"+ str(regr.coef_)
#plt.scatter(dates_X_test, stock_y_test,  color='orange')
f, ax = plt.subplots(1, 1)
sns.set()
ax.plot(str_age_df[fit_colmnslst[1]],\
        str_age_df[by_age[len(by_age)-1]], color='black',linestyle="--")
ax.scatter(poly_X_test.loc[:,fit_colmnslst[1]], \
           poly_y_test,  color='purple',label="Actual value")
ax.scatter(poly_X_test.loc[:,fit_colmnslst[1]],\
        poly_y_pred, color='black',linestyle="--",label="Polynomial Reg.")
plt.show()
print '+'*space
print"Cross validation score: "
scores = cross_val_score(regr, poly_X_train, poly_y_train,scoring="neg_mean_squared_error", cv=len(poly_X_train))
print scores
print '+'*space
rmse_scores = np.sqrt(-scores)
print "Root mean square error: "+str(rmse_scores)
display_scores(scores)
print '+'*space
print cross_val_score(regr, str_age_df[fit_colmnslst],str_age_df[by_age[len(by_age)-1]],scoring='neg_mean_squared_error')
#scores = cross_val_score(clf, poly_X_train, poly_y_train,scoring="neg_mean_squared_error", cv=10)
#ax.legend()
#plt.xlabel("Features = "+str(list(output_ply_diab_df.columns)[whch_fetu]))
#plt.ylabel("Disease progression") 

print "Pre cross validation score:", lin_model.score(poly_X_test, poly_y_test)
scores = cross_val_score(lin_model, str_age_df[fit_colmnslst], str_age_df[by_age[len(by_age)-1]], cv=4)
# Make cross validated predictions
predictions = cross_val_predict(lin_model, str_age_df[fit_colmnslst], str_age_df[by_age[len(by_age)-1]], cv=4)
plt.scatter(str_age_df[by_age[len(by_age)-1]], predictions)
plt.show()
print "Post cross-validation scores:", scores
accuracy = metrics.r2_score(str_age_df[by_age[len(by_age)-1]], predictions)
print "Cross-Predicted Accuracy:", accuracy